In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime
import pandas as pd
import plotly.express as px
import time
import copy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.simplefilter("ignore")

## 1. ETL (Extract, transform, Load)

In [3]:
df = pd.read_csv('datasets/selected_2000user_v2.csv')
df

,MeterID,DATE-Time,KWH,user
0,6200093839,2021-01-01 00:00:00,0.602,0
1,6200093839,2021-01-01 00:15:00,0.589,0
2,6200093839,2021-01-01 00:30:00,0.410,0
3,6200093839,2021-01-01 00:45:00,0.495,0
4,6200093839,2021-01-01 01:00:00,0.450,0
...,...,...,...,...
40181914,6200051044,2021-07-31 22:45:00,0.473,2000
40181915,6200051044,2021-07-31 23:00:00,0.461,2000
40181916,6200051044,2021-07-31 23:15:00,0.445,2000
40181917,6200051044,2021-07-31 23:30:00,0.460,2000


outliers = pd.read_csv('datasets/noise_users.csv')
outliers

In [5]:
df = df[df['user'].isin(outliers['Series'])]
print(df['user'].nunique())
df

163


,MeterID,DATE-Time,KWH,user
109288,6200063683,2021-01-01 00:00:00,0.000,5
109289,6200063683,2021-01-01 00:15:00,0.000,5
109290,6200063683,2021-01-01 00:30:00,0.000,5
109291,6200063683,2021-01-01 00:45:00,0.000,5
109292,6200063683,2021-01-01 01:00:00,0.000,5
...,...,...,...,...
40023994,6200050861,2021-07-31 22:45:00,0.032,1992
40023995,6200050861,2021-07-31 23:00:00,0.034,1992
40023996,6200050861,2021-07-31 23:15:00,0.015,1992
40023997,6200050861,2021-07-31 23:30:00,0.014,1992


## 2. EDA (Exploratory Data Analysis)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3258416 entries, 109288 to 40023998
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   MeterID    int64  
 1   DATE-Time  object 
 2   KWH        float64
 3   user       int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 124.3+ MB


In [ ]:
# Updated
def clean_data(df, user_id):
    # Select the data for the specified user
    user_data = df[df['user']==int(user_id)]

    # Drop not useful columns
    user_data['DATE-Time'] = pd.to_datetime(user_data['DATE-Time'], infer_datetime_format=True)
    user_data.drop(columns=["MeterID", "user"], inplace=True)

    # Resample `15min` in order to have completed `"DATE-Time"`
    # I have `'DATE-Time'` 32760 rows (32760/96=341.25 Days)
    # It should be (365day * 96point = 35040 rows)
    # I have to reset index in order to hava 35040 rows
    # Fill the missing value
    user_data = user_data.set_index('DATE-Time').resample("15T").mean()
    
    # Convert the index to a DatetimeIndex object
    user_data.index = pd.to_datetime(user_data.index)

    # Count the number of zero values in each day
    zero_counts = user_data.groupby(user_data.index.floor('D'))['KWH'].apply(lambda x: (x == 0).sum())

    # Get the dates where the number of zero values is less than or equal to 48
    valid_dates = zero_counts[zero_counts <= 48].index

    # Drop the dates where the number of zero values is greater than 48
    user_data = user_data[user_data.index.floor('D').isin(valid_dates)]

    # Drop the days that have NaN values
    # Check which rows have at least one missing value
    missing_days = user_data.isnull().any(axis=1)

    # Get the dates (i.e., indices) of the rows that have missing values
    missing_days_index = missing_days[missing_days].index

    # Get the unique dates that have missing values
    missing_dates = pd.Series(missing_days_index.date).unique().tolist()

    # Drop the entire dates that contain Nan values
    user_data_clean = user_data[~user_data.index.to_series().dt.date.isin(missing_dates)]
    
    if user_data_clean.empty:
        return None

    return user_data_clean